In [13]:
import tensorflow as tf
from PIL import Image
from tensorflow.keras import layers,models
import matplotlib.pyplot as plt
import numpy as np

In [14]:
Batch_Size=38
Image_Size=256
Channels=3
Epochs=50

In [15]:
dataset= tf.keras.preprocessing.image_dataset_from_directory(
    "D:\VSCode\AI & ML Project\Dataset\Corn",
    shuffle=True,
    image_size=(Image_Size,Image_Size),
    batch_size= Batch_Size)


Found 3852 files belonging to 4 classes.


In [16]:
class_names= dataset.class_names
class_names

['Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy']

In [17]:
def get_dataset_split(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
    assert(train_split+val_split+test_split)==1
    ds_size= len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=10)
    train_size=int(train_split*ds_size)
    val_size=int(val_split*ds_size)
    train_ds=ds.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    
    return train_ds,test_ds,val_ds

In [18]:
train_ds,test_ds,val_ds = get_dataset_split(dataset)

In [19]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)#loading to cache memory to reduce training time
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [20]:
resize_and_rescale =tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(Image_Size,Image_Size),#resizing
    layers.experimental.preprocessing.Rescaling(1.0/255)#normalizing
]
)

In [21]:
data_augmentation= tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomContrast(0.2),
    layers.experimental.preprocessing.RandomZoom(0.3)
])

In [22]:
input_shape=(Batch_Size,Image_Size,Image_Size,Channels)
n_classes=4
model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(64,kernel_size=(3,3),activation='relu',input_shape=input_shape),#relu makes model non linear by converting negative values in feature map to 0
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax'),
])
model.build(input_shape=input_shape)

In [23]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [24]:
data=model.fit(
    train_ds,
    batch_size=Batch_Size,
    validation_data=val_ds,
    verbose=1,
    epochs=Epochs)

Epoch 1/50
81/81 [==============================] - 205s 2s/step - loss: 0.9507 - accuracy: 0.5766 - val_loss: 0.3563 - val_accuracy: 0.8511
Epoch 2/50
81/81 [==============================] - 197s 2s/step - loss: 0.4082 - accuracy: 0.8317 - val_loss: 0.4083 - val_accuracy: 0.8427
Epoch 3/50
81/81 [==============================] - 192s 2s/step - loss: 0.4399 - accuracy: 0.8307 - val_loss: 0.2768 - val_accuracy: 0.8792
Epoch 4/50
81/81 [==============================] - 191s 2s/step - loss: 0.3252 - accuracy: 0.8563 - val_loss: 0.2458 - val_accuracy: 0.8652
Epoch 5/50
81/81 [==============================] - 191s 2s/step - loss: 0.3019 - accuracy: 0.8680 - val_loss: 0.2487 - val_accuracy: 0.8792
Epoch 6/50
81/81 [==============================] - 199s 2s/step - loss: 0.2804 - accuracy: 0.8713 - val_loss: 0.2266 - val_accuracy: 0.8792
Epoch 7/50
81/81 [==============================] - 190s 2s/step - loss: 0.2963 - accuracy: 0.8677 - val_loss: 0.2351 - val_accuracy: 0.8961
Epoch 8/50
81

In [25]:
scores=model.evaluate(test_ds)

11/11 [==============================] - 7s 482ms/step - loss: 0.0922 - accuracy: 0.9594


In [26]:
model.save("D:/VSCode/AI & ML Project/models/corn")

INFO:tensorflow:Assets written to: D:/VSCode/AI & ML Project/models/corn\assets


INFO:tensorflow:Assets written to: D:/VSCode/AI & ML Project/models/corn\assets


In [27]:
model=tf.keras.models.load_model("D:/VSCode/AI & ML Project/models/corn")

In [28]:
# for images_batch,labels_batch in test_ds.take(1):
#     image=images_batch[0].numpy().astype("uint8")
#     plt.imshow(image)
   
filename = "D:/VSCode/AI & ML Project/uploads/1fecd8b3-b9a7-4d56-97f4-9728c20d0877___Rutg._Bact.S 1632.JPG"
def process_image(image_path):
    # Load and preprocess the image for your model
    img = Image.open(image_path)
    img = img.resize((256, 256))  # adjust the size according to your model's input
    img_array = np.array(img) / 255.0  # normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
    return img_array

img_array = process_image(filename)
batch_prediction= model.predict(img_array)
print("Predicted Disease",class_names[np.argmax(batch_prediction[0])])

1/1 [==============================] - 0s 140ms/step
Predicted Disease Corn_(maize)___Common_rust_
